# Importing Jornada geodatabases to jgeo

This script was used to import geodatabases to the jgeo PostGIS instance. Each geodatabase
was handled slightly differently since sometimes there are errors with a layer. Once the
layers were in the schema they were renamed and moved to present locations.

Basically uses GDAL (ogr2ogr) to put ESRI File Geodatabase layers (gdb) into an
existing PostGIS schema.

In [1]:
import fiona
import os
import sys
# Get credentials for jgeo
sys.path.append('/home/greg/admin')
import jgeo_cred
host=jgeo_cred.host
user=jgeo_cred.user
password=jgeo_cred.pwd
database=jgeo_cred.db
# Path to geodatabases
jgdb_path = '/home/greg/data/rawdata/JornadaGIS_copies'
logs_path = '/home/greg/data/rawdata/JornadaGeospatial/logs/'

In [7]:
# The ogr2ogr (GDAL) commands to use to place the layer in gjeo
# Output is logged from the operation
cmdBase = 'ogr2ogr -append --config OGR_TRUNCATE YES --config PG_USE_COPY YES \
    -nlt PROMOTE_TO_MULTI \
    -nln {schema}.{table} \
    -f "PostgreSQL" PG:"host={host} user={user} password={password} dbname={database}" \
    "{inputFile}" "{layerName}" 2>&1 | tee {logname}'

cmdBase_overmulti = 'ogr2ogr -overwrite --config OGR_TRUNCATE YES --config PG_USE_COPY YES \
    -nlt MULTIPOLYGON \
    -nln {schema}.{table} \
    -f "PostgreSQL" PG:"host={host} user={user} password={password} dbname={database}" \
    "{inputFile}" "{layerName}" 2>&1 | tee {logname}'

# One for appending misc files
# The ogr2ogr (GDAL) commands to use to place the layer in gjeo
# Output is logged from the operation
cmdBase_append = 'ogr2ogr -append --config OGR_TRUNCATE YES --config PG_USE_COPY YES \
    -nlt PROMOTE_TO_MULTI \
    -nln {schema}.{table} \
    -f "PostgreSQL" PG:"host={host} user={user} password={password} dbname={database}" \
    "{inputFile}" "{layerName}" 2>&1 | tee -a {logname}'


# Function to run the command over a list of layers
def run_cmd(layerlist, command, inputFile, schema, host,
            user, password, database, logname):
    for layerName in layerlist:
        #new table name corresponds to layerName. You can change it here
        table=layerName
        # Format the base command with inputfile, schema, etc
        cmd = command.format(inputFile=inputFile, schema=schema, table=table,
                             host=host, user=user, password=password, database=database,
                             layerName=layerName, logname=logname)
        print(cmd)
        #run system command
        os.system(cmd)
        #print(out)

## Import the "All_studies" jornada geodatabase to jgeo

In [4]:
# Select the Geodatabase and target schema in jgeo
# Note that the schema must be present in jgeo first!
inputFile = os.path.join(jgdb_path, 'All_studies_JornadaGDB_03_30_21.gdb')
schema='latest_all_jornada_studies_gdb'
logname1 = os.path.join(logs_path, 'JornadaGDB_log.txt')
logname2 = os.path.join(logs_path, 'JornadaGDB_extra_log.txt')
#srs='EPSG:4326'

In [3]:
fiona.listlayers(inputFile)

['JornadaStudy_009_animal_transects',
 'JornadaStudy_010_termite_bait',
 'JornadaStudy_012_transect_hydroprobe_tubes',
 'JornadaStudy_013_npp_hydroprobe_tubes',
 'JornadaStudy_120_boundary_fence_intercept_plant_lines',
 'JornadaStudy_121_biodiversity_plots',
 'JornadaStudy_122_crust_abrasion_scrape_site_bsne',
 'JornadaStudy_126_lter_weather_station',
 'JornadaStudy_127_evaporation_pan',
 'JornadaStudy_129_upper_trailer_tipping_bucket_raingauge',
 'JornadaStudy_169_npp_adc',
 'JornadaStudy_175_instrumented_stock_ponds',
 'JornadaStudy_392_lagomorph_creostebush_plot',
 'JornadaStudy_262_ecotone_rodent_satellite_grid',
 'JornadaStudy_262_rodent_ecotone_grid',
 'JornadaStudy_268_npp_tipping_bucket_raingage',
 'JornadaStudy_278_hydroprobe_tubes',
 'JornadaStudy_287_npp_phenology_transects',
 'JornadaStudy_288_npp_geomet_bsne',
 'JornadaStudy_301_wind_driver_decomposition',
 'JornadaStudy_343_multiple_stressor_experiment_erosion_pin',
 'JornadaStudy_351_permanent_chart_quadrat',
 'JornadaSt

In [ ]:
# Run the command over the layers in the inputFile gdb
run_cmd(fiona.listlayers(inputFile), cmdBase,
        inputFile, schema, host, user, password, database,
        logname1)

In [41]:
# Check the logs and for those that failed
failedlayers = ["JornadaStudy_413_CSIS_OuterBdry_3mBuf","JornadaStudy_413_CSIS_IntermedLyrs_CSIS_BlocksBuf3m",
                "JornadaStudy_188_CollectionArea"]
run_cmd(failedlayers, cmdBase_overmulti, # Overwrite with multipoly command
        inputFile, schema, host, user, password, database,
        logname2)


## Import the "Infrastructure..." jornada geodatabase to jgeo

In [79]:
# Select the Geodatabase and target schema in jgeo
# Note that the schema must be present in jgeo first!
inputFile = os.path.join(jgdb_path, 'Infrastructure2_21Sep15.gdb')
schema='infrastructure_two_gdb'
logname1 = os.path.join(logs_path, 'JornadaGDB_Infrastructure2_log.txt')
logname2 = os.path.join(logs_path, 'JornadaGDB_Infrastructure2_extra_log.txt')

In [11]:
fiona.listlayers(inputFile)

['Airstrip',
 'All_RainGauge_15Oct13',
 'Cattleguards',
 'CDRRC_Exclosures_17Sep15',
 'ClosedRoad',
 'DefunctTowers',
 'Gates',
 'hwybordrXing',
 'IsaackLakePoly',
 'RemvdFenceLabel',
 'RemvdFences',
 'RioGrande',
 'UTEP_Site_Access_Trail_UTM',
 'Tanks',
 'Towers3',
 'Wells',
 'RanchStructures',
 'CDRRC_Bdry_25Aug15',
 'CDNP_Bdry_25Aug15',
 'JER_Bdry_25Aug15',
 'ExclosuresJER_17Jul2015',
 'OffLimits_20Aug15',
 'All3_Bdry_25Aug15',
 'Fence_CDRRC_26Aug15',
 'Fence_JER_28Aug15',
 'Lvstk_Corral_Lot_Lane',
 'NPP_Sites_study011',
 'NPP_Sites_study011_Centroid',
 'Pastures_JER_16Sep15',
 'DOQQ_List_JER',
 'Pastures_CDRRC2_16Sep15',
 'DOQs_List_JER',
 'All_RG_25Mar15_WGS84',
 'All5_Sites_420m_Buf50',
 'All5_Sites_420m_Landsat_poly',
 'BurnPlots21Aug13',
 'CDRRC_ExcloEDITED21Aug13',
 'ExclusuresJER_Jul172015',
 'JER_29Rgs',
 'JER_CDRRC_Bdry_FromCorrectedMar2014',
 'JER_DeededBndry_OCt2013',
 'JER_FtB_WSMR_bndry',
 'JER_Rg_DThatcher_Oct2013',
 'OffLimits_27Feb2014_Final',
 'StudiesBuf50_26Nov201

In [ ]:
# Run the command over the layers in the inputFile gdb
run_cmd(fiona.listlayers(inputFile), cmdBase,
        inputFile, schema, host, user, password, database,
        logname1)

In [80]:
# Check the logs and for those that failed
failedlayers = ["IsaackLakePoly","All5_Sites_420m_Buf50","All5_Sites_420m_Landsat_poly","OffLimits_27Feb2014_Final",
               "StudiesBuf50_26Nov2014",]
run_cmd(failedlayers, cmdBase_overmulti, # Overwrite with multipoly command
        inputFile, schema, host, user, password, database,
        logname2)

## Import the "NonResearchLayers..." jornada geodatabase to jgeo

In [4]:
# Select the Geodatabase and target schema in jgeo
# Note that the schema must be present in jgeo first!
inputFile = os.path.join(jgdb_path, 'NonResearchLyrs_17Sep14.gdb')
schema='non_research_layers_gdb'
logname1 = os.path.join(logs_path, 'JornadaGDB_NonResearch_log.txt')
logname2 = os.path.join(logs_path, 'JornadaGDB_NonResearch_extra_log.txt')

In [5]:
fiona.listlayers(inputFile)

['Grass_Cover_UTM',
 'Soilmu_a_nm690',
 'Soilmu_a_nm719',
 'JERGrassCondition1858',
 'JERShrubPresence1858',
 'JERVegetation1915',
 'JERVegetation1928',
 'JERVegetation1998']

In [6]:
# Run the command over the layers in the inputFile gdb
run_cmd(fiona.listlayers(inputFile), cmdBase,
        inputFile, schema, host, user, password, database,
        logname1)

ogr2ogr -append --config OGR_TRUNCATE YES --config PG_USE_COPY YES     -nlt PROMOTE_TO_MULTI     -nln non_research_layers_gdb.Grass_Cover_UTM     -f "PostgreSQL" PG:"host=termite user=gmaurer password=pgDelaplaya!71 dbname=jgeo"     "/home/greg/data/rawdata/JornadaGIS_copies/NonResearchLyrs_17Sep14.gdb" "Grass_Cover_UTM" 2>&1 | tee /home/greg/data/rawdata/JornadaGeospatial/logs/JornadaGDB_NonResearch_log.txt
Warning 1: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
ogr2ogr -append --config OGR_TRUNCATE YES --config PG_USE_COPY YES     -nlt PROMOTE_TO_MULTI     -nln non_research_layers_gdb.Soilmu_a_nm690     -f "PostgreSQL" PG:"host=termite user=gmaurer password=pgDelaplaya!71 dbname=jgeo"     "/home/greg/data/rawdata/JornadaGIS_copies/

In [77]:
# Check the logs and for those that failed
failedlayers = []
run_cmd(failedlayers, cmdBase_overmulti, # Overwrite with multipoly command
        inputFile, schema, host, user, password, database,
        logname2)

## Import the "JornadaStudiesCleaned..." jornada geodatabase to jgeo

In [6]:
# Select the Geodatabase and target schema in jgeo
# Note that the schema must be present in jgeo first!
inputFile = os.path.join(jgdb_path, 'Jornada_Studies_Cleaned.gdb')
schema='jornada_studies_cleaned_gdb'
logname1 = os.path.join(logs_path, 'JornadaGDB_StudiesCleaned_log.txt')
logname2 = os.path.join(logs_path, 'JornadaGDB_StudiesCleaned_extra_log.txt')

In [7]:
fiona.listlayers(inputFile)

['JornadaStudy_001_hydrology_runoff_gps_points',
 'JornadaStudy_001_Hydrology_Runoff_Drums',
 'JornadaStudy_001_Hydrology_Runoff_Exclosures',
 'JornadaStudy_001_Hydrology_Runoff_Plates',
 'JornadaStudy_001_Hydrology_Runoff_Plots',
 'JornadaStudy_001_hydrology_runoff_plot_chemistry_data',
 'JornadaStudy_001_hydrology_runoff_plot_runoff_data',
 'JornadaStudy_001_hydrology_runoff_plot_veg_cover',
 'JornadaStudy_005_small_mammal_traps',
 'JornadaStudy_003_transect_direct_reading_raingauge',
 'JornadaStudy_003_transect_precip_weekly',
 'JornadaStudy_009_animal_transects',
 'JornadaStudy_009_Transects',
 'JornadaStudy_009_animal_tansect_data',
 'JornadaStudy_007_lizard_pitfall_data',
 'JornadaStudy_007_lizard_pitfall_traps',
 'JornadaStudy_008_npp_arthropod_pitfall_traps_1988_1994',
 'JornadaStudy_008_npp_arthropod_pitfall_traps_1996_2000',
 'JornadaStudy_008_npp_arthropod_1996_2000',
 'JornadaStudy_008_npp_arthropod_trap_plots',
 'JornadaStudy_008_npp_arthropod_1988_1994',
 'JornadaStudy_01

In [ ]:
run_cmd(fiona.listlayers(inputFile), cmdBase,
        inputFile, schema, host, user, password, database,
        logname1)

In [77]:
# Check the logs and for those that failed
failedlayers = []
run_cmd(failedlayers, cmdBase_overmulti, # Overwrite with multipoly command
        inputFile, schema, host, user, password, database,
        logname2)

## Import various layers

In [14]:
# Select the Geodatabase and target schema in jgeo
# Note that the schema must be present in jgeo first!
inputFile = os.path.join('/home/greg/Downloads/', 'NM690/spatial')
schema='biogeography_and_soils'
logname1 = os.path.join(logs_path, 'JornadaGDB_Misc_log.txt')
logname2 = os.path.join(logs_path, 'JornadaGDB_Misc_extra_log.txt')

In [15]:
fiona.listlayers(inputFile)

['soilsa_a_nm690',
 'soilmu_a_nm690',
 'soilsf_p_nm690',
 'soilmu_p_nm690',
 'soilmu_l_nm690',
 'soilsf_l_nm690']

In [18]:
run_cmd(fiona.listlayers(inputFile), cmdBase_append, # Use the append commend to keep a running log
        inputFile, schema, host, user, password, database,
        logname1)

ogr2ogr -append --config OGR_TRUNCATE YES --config PG_USE_COPY YES     -nlt PROMOTE_TO_MULTI     -nln biogeography_and_soils.soilsa_a_nm690     -f "PostgreSQL" PG:"host=termite user=gmaurer password=pgDelaplaya!71 dbname=jgeo"     "/home/greg/Downloads/NM690/spatial" "soilsa_a_nm690"
ogr2ogr -append --config OGR_TRUNCATE YES --config PG_USE_COPY YES     -nlt PROMOTE_TO_MULTI     -nln biogeography_and_soils.soilmu_a_nm690     -f "PostgreSQL" PG:"host=termite user=gmaurer password=pgDelaplaya!71 dbname=jgeo"     "/home/greg/Downloads/NM690/spatial" "soilmu_a_nm690"


ERROR 1: COPY statement failed.
ERROR:  null value in column "ogc_fid" of relation "soilmu_a_nm690" violates not-null constraint
DETAIL:  Failing row contains (null, 0106000020AD1000000100000001030000000100000015000000437ACBC833AC..., NM690, 3, Ge, 634564, null, null).
CONTEXT:  COPY soilmu_a_nm690, line 1: "0106000020AD1000000100000001030000000100000015000000437ACBC833AC5AC0CDBBAB0947104040E46487B431AC5AC0..."



ogr2ogr -append --config OGR_TRUNCATE YES --config PG_USE_COPY YES     -nlt PROMOTE_TO_MULTI     -nln biogeography_and_soils.soilsf_p_nm690     -f "PostgreSQL" PG:"host=termite user=gmaurer password=pgDelaplaya!71 dbname=jgeo"     "/home/greg/Downloads/NM690/spatial" "soilsf_p_nm690"
ogr2ogr -append --config OGR_TRUNCATE YES --config PG_USE_COPY YES     -nlt PROMOTE_TO_MULTI     -nln biogeography_and_soils.soilmu_p_nm690     -f "PostgreSQL" PG:"host=termite user=gmaurer password=pgDelaplaya!71 dbname=jgeo"     "/home/greg/Downloads/NM690/spatial" "soilmu_p_nm690"
ogr2ogr -append --config OGR_TRUNCATE YES --config PG_USE_COPY YES     -nlt PROMOTE_TO_MULTI     -nln biogeography_and_soils.soilmu_l_nm690     -f "PostgreSQL" PG:"host=termite user=gmaurer password=pgDelaplaya!71 dbname=jgeo"     "/home/greg/Downloads/NM690/spatial" "soilmu_l_nm690"
ogr2ogr -append --config OGR_TRUNCATE YES --config PG_USE_COPY YES     -nlt PROMOTE_TO_MULTI     -nln biogeography_and_soils.soilsf_l_nm690     -